In [4]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import KNNImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd


In [14]:
data_path='data/data.csv'
# Reload the data to start fresh
data = pd.read_csv(data_path)

# Convert 'date' to datetime

data['date'] = pd.to_datetime(data['date'], infer_datetime_format=True)

# Set date as index


data = data.set_index('date').sort_index()

/var/folders/dk/djl_jn195h7gym2_kn27310c0000gn/T/ipykernel_7933/3925681534.py:7: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data['date'] = pd.to_datetime(data['date'], infer_datetime_format=True)


In [15]:
# Initialize KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)

# Separate the last 168 rows for prediction
data_for_imputation = data.iloc[:-168, :]
data_to_predict = data.iloc[-168:, :]

# Impute missing values in 'exog' for the whole dataset
data['exog'] = knn_imputer.fit_transform(data[['exog']])

# Impute missing values in 'power' except for the last 168 values
data_for_imputation['power'] = knn_imputer.fit_transform(data_for_imputation[['power']])

# Combine the data back
data_imputed = pd.concat([data_for_imputation, data_to_predict])

# Check if there are any missing values left in 'exog' and 'power' (except for the last 168 values)
data_imputed.isnull().sum(), data_imputed.tail()

/var/folders/dk/djl_jn195h7gym2_kn27310c0000gn/T/ipykernel_7933/115459068.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_imputation['power'] = knn_imputer.fit_transform(data_for_imputation[['power']])


(power    168
 exog     259
 dtype: int64,
                      power    exog
 date                              
 2023-10-15 19:00:00    NaN  10.820
 2023-10-15 20:00:00    NaN  10.619
 2023-10-15 21:00:00    NaN   9.535
 2023-10-15 22:00:00    NaN  10.031
 2023-10-15 23:00:00    NaN   9.464)

In [16]:








###########
# Define the feature engineering function
def build_some_features(df_, target, num_periods_lagged=1, num_periods_diffed=0, weekday=False, month=False, rolling=[], holidays=False):
    """
    Builds some features by calculating differences between periods.
    """
    df_ = df_.copy()
            
    for i in range(1, num_periods_lagged+1):
        df_['lagged_%s' % str(i)] = df_[target].shift(i)
        
    for i in range(1, num_periods_diffed+1):
        df_['diff_%s' % str(i)] = df_[target].diff(i)
    
    for stat in rolling:
        if stat:
            df_['rolling_mean'] = df_[target].rolling('7D').mean()
            df_['rolling_min'] = df_[target].rolling('7D').min()
            df_['rolling_max'] = df_[target].rolling('7D').max()
            df_['rolling_std'] = df_[target].rolling('7D').std()
        
    if weekday:
        df_['sin_weekday'] = np.sin(2 * np.pi * df_.index.weekday / 7)
        df_['cos_weekday'] = np.cos(2 * np.pi * df_.index.weekday / 7)
        
    if month:
        df_['sin_month'] = np.sin(2 * np.pi * df_.index.month / 12)
        df_['cos_month'] = np.cos(2 * np.pi * df_.index.month / 12)
        
    if holidays:
        holidays = df_[((df_.index.month == 12) & (df_.index.day == 25))
                       | ((df_.index.month == 1) & (df_.index.day == 1))][target]
        df_['holidays'] = holidays + 1
        df_['holidays'].fillna(0, inplace=True)
        
    df_.fillna(method='ffill', inplace=True)
    
    return df_

# Apply feature engineering
features_to_roll = [np.mean, np.min, np.max, np.std]
data_fe = build_some_features(data, 'power', num_periods_lagged=3, num_periods_diffed=1, weekday=True, month=True, rolling=features_to_roll, holidays=True)

# Drop NA values generated by lag/diff operations
data_fe.dropna(inplace=True)

# Preview the engineered features
data_fe.head()


/var/folders/dk/djl_jn195h7gym2_kn27310c0000gn/T/ipykernel_7933/1762420660.py:36: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_.fillna(method='ffill', inplace=True)


,power,exog,lagged_1,lagged_2,lagged_3,diff_1,rolling_mean,rolling_min,rolling_max,rolling_std,sin_weekday,cos_weekday,sin_month,cos_month,holidays
date,,,,,,,,,,,,,,,
2020-09-20 03:00:00,984.234,11.025,928.813,952.705,976.930,55.421,960.670500,928.813,984.234,25.152597,-0.781831,0.62349,-1.0,-1.836970e-16,0.0
2020-09-20 04:00:00,1002.113,11.196,984.234,928.813,952.705,17.879,968.959000,928.813,1002.113,28.600455,-0.781831,0.62349,-1.0,-1.836970e-16,0.0
2020-09-20 05:00:00,996.167,11.414,1002.113,984.234,928.813,-5.946,973.493667,928.813,1002.113,27.888492,-0.781831,0.62349,-1.0,-1.836970e-16,0.0
2020-09-20 06:00:00,970.203,11.382,996.167,1002.113,984.234,-25.964,973.023571,928.813,1002.113,25.488957,-0.781831,0.62349,-1.0,-1.836970e-16,0.0
2020-09-20 07:00:00,890.502,10.974,970.203,996.167,1002.113,-79.701,962.708375,890.502,1002.113,37.524668,-0.781831,0.62349,-1.0,-1.836970e-16,0.0


In [22]:
from sklearn.metrics import mean_absolute_error

# Preparing the data for training and prediction
# Drop rows with NaN in the target column 'power' for training data
train_df = data_fe.iloc[:-168].dropna(subset=['power'])
predict_df = data_fe.iloc[-168:]  # Data for prediction

# Define features and target
X_train = train_df.drop('power', axis=1)
y_train = train_df['power']
X_predict = predict_df.drop('power', axis=1)




In [26]:
from sklearn.metrics import mean_absolute_error

# Preparing the data for training and prediction
# Drop rows with NaN in the target column 'power' for training data
train_df = data_fe.iloc[:-168].dropna(subset=['power'])
predict_df = data_fe.iloc[-168:]  # Data for prediction

# Define features and target
X_train = train_df.drop('power', axis=1)
y_train = train_df['power']
X_predict = predict_df.drop('power', axis=1)

# Initialize the GradientBoostingRegressor model
gb_regressor = GradientBoostingRegressor()

# Train the model
gb_regressor.fit(X_train, y_train)

# Predict the last 168 missing values of 'power'
power_predictions = gb_regressor.predict(X_predict)

# As we don't have true values for the last 168 hours, we skip direct MAE calculation.
# Normally, you'd calculate it like this if you had true values:
# mae = mean_absolute_error(y_true, power_predictions)

# Insert the predicted values back into the dataset
data_fe.loc[data_fe.index[-168:], 'power'] = power_predictions

# Display the first few predicted values
power_predictions[:5]


array([710.53153097, 710.53153097, 710.53153097, 710.53153097,
       710.53153097])

In [27]:
# Apply forward fill to handle NaNs resulting from feature engineering
data_fe_filled = data_fe.ffill().bfill()  # Forward fill followed by backward fill to cover all NaNs

# Prepare the data again for training and prediction
X_train_filled = data_fe_filled.iloc[:-168].drop('power', axis=1)  # Exclude last 168 hours for features
y_train_filled = data_fe_filled.iloc[:-168]['power']  # Exclude last 168 hours for target
X_predict_filled = data_fe_filled.iloc[-168:].drop('power', axis=1)  # Features for prediction

# Re-initialize and train the GradientBoostingRegressor model
gb_regressor = GradientBoostingRegressor()
gb_regressor.fit(X_train_filled, y_train_filled)

# Re-predict the last 168 missing values of 'power'
power_predictions_filled = gb_regressor.predict(X_predict_filled)

# Insert the predicted values back into the dataset
data_fe_filled.loc[data_fe_filled.index[-168:], 'power'] = power_predictions_filled

# Display the first few predicted values as a check
power_predictions_filled[:5]



array([710.53153097, 710.53153097, 710.53153097, 710.53153097,
       710.53153097])

In [25]:
rmse_gb = mean_squared_error(y_train, y_pred_train, squared=False)

rmse_gb

mae_gb = mean_absolute_error(y_train, y_pred_train)

ValueError: Found input variables with inconsistent numbers of samples: [26733, 168]

In [20]:
rmse_gb,mae_gb

(11.994093059455155, 8.597240482209516)

In [21]:
len(y_pred_train)

168